In [7]:
from dotenv import dotenv_values
import json

In [10]:
import sys
!{sys.executable} -m pip install cohere

  Preparing metadata (setup.py) ... done
  Created wheel for cohere: filename=cohere-3.1.6-cp39-cp39-macosx_10_9_x86_64.whl size=11873 sha256=2a7a837e07b5efd3099e7b06e2cb66dc4370dc64bac3527b58676de0820de54e
  Stored in directory: /Users/jasonwang/Library/Caches/pip/wheels/15/44/15/a2acb5fc0a17bf3d0515c5ccd06ad6e4b3dba4e7201e870d39
Successfully built cohere

[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [8]:
config = dotenv_values(".env")

In [13]:
CATEGORIES = [
    'american',
    'asian',
    'chinese',
    'european',
    'french',
    'hamburgers',
    'healthy',
    'italian',
    'japanese',
    'korean',
    'mediterranean',
    'mexican',
    'thai',
    'dessert',
    'coffee'
]

In [14]:
len(CATEGORIES)

15

In [17]:
import cohere  
co = cohere.Client(config['COHERE_API_KEY'])

In [37]:
from cohere.classify import Example
examples = []
# load in labelled dataset
with open('data/all_restaurants_details_labelled.json', 'r') as f:
    labelled_locations = json.load(f)

for location in labelled_locations:
    examples.append(Example(location['editorial_summary']['overview'], location['category']))
    examples.append(Example(location['name'], location['category']))

In [38]:
examples

[Example(text='Eatery with a bustling patio & bright, snug interior, serving classic Mexican fare & margaritas.', label='mexican'),
 Example(text='El Trompo', label='mexican'),
 Example(text="Susur Lee's stylish eatery for creative Asian & French-influenced dishes meant for sharing.", label='asian'),
 Example(text='Lee', label='asian'),
 Example(text='Canadian chain selling signature premium-blend coffee, plus light fare like pastries & panini.', label='coffee'),
 Example(text='Tim Hortons', label='coffee'),
 Example(text="Vietnamese quick-serve restaurant with a health focus that's open late for noodles & sandwiches.", label='asian'),
 Example(text='Ginger', label='asian'),
 Example(text='Fast-food outlet serving up namesake root beer, plus burgers, chicken & fries.', label='hamburgers'),
 Example(text='A&W Canada', label='hamburgers'),
 Example(text='Intimate eatery for Eritrean & Ethiopian eats, with vegan & gluten-free options & a coffee ceremony.', label='african'),
 Example(text=

In [20]:
with open('data/all_restaurants_details.json', 'r') as f:
    unlabelled_locations = json.load(f)

In [21]:
len(unlabelled_locations)

763

In [26]:
# extract all summaries
test_data = {}
for i in range(len(unlabelled_locations)):
    try:
        test_data[i] = unlabelled_locations[i]['editorial_summary']['overview']
    except KeyError:
        test_data[i] = unlabelled_locations[i]['name']

In [28]:
len(test_data)

763

In [39]:
res = []
for i in range(len(test_data)//96):
    inputs = list(test_data.values())[i*96:(i+1)*96]
    response = co.classify(
        model = 'large',
        inputs = inputs,
        examples = examples
    )
    res += response.classifications

In [44]:
len(res)

672

In [45]:
for i in range(len(res)):
    unlabelled_locations[i]['category'] = res[i].prediction

In [46]:
with open('data/all_restaurants_labelled_cohere.json', 'w') as f:
    json.dump(unlabelled_locations, f)